In [2]:
%pip -q install --upgrade "transformers" "accelerate" "peft" "bitsandbytes" "datasets" "huggingface_hub" "sentencepiece"
!pip install trl

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 15.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 506.8/506.8 kB 47.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 MB 21.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pylibcudf-cu12 25.6.0 requires pyarrow<20.0.0a0,>=14.0.0; platform_machine == "x86_64", but you have pyarrow 22.0.0 which is incompatible.
cudf-cu12 25.6.0 requires pyarrow<20.0.0a0,>=14.0.0; platform_machine == "x86_64", but you have pyarrow 22.0.0 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 423.1/423.1 kB 14.7 MB/s eta 0:00:00


In [2]:
from huggingface_hub import login
token = "Your huggingface Token"

In [1]:
import torch, os
from datasets import load_dataset
from transformers import (
    AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, TrainingArguments
)
from peft import LoraConfig, prepare_model_for_kbit_training
from trl import SFTTrainer

MODEL_ID = "Qwen/Qwen2.5-1.5B-Instruct"

is_bf16 = torch.cuda.is_bf16_supported()
compute_dtype = torch.bfloat16 if is_bf16 else torch.float16

attn_impl = "sdpa" if torch.__version__ >= "2.1" else "eager"
if "T4" in torch.cuda.get_device_name(0):
    attn_impl = "eager"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=compute_dtype,
)

tokenizer = AutoTokenizer.from_pretrained(MODEL_ID, use_fast=True, token = auth)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [10]:
import bitsandbytes
print(bitsandbytes.__version__)

0.48.2


In [2]:

model = AutoModelForCausalLM.from_pretrained(
    MODEL_ID,
    quantization_config=bnb_config,
    attn_implementation=attn_impl,
    torch_dtype=compute_dtype,
    device_map="auto",
)

model.config.use_cache = False
model = prepare_model_for_kbit_training(model)


`torch_dtype` is deprecated! Use `dtype` instead!


model.safetensors:   0%|          | 0.00/3.09G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/242 [00:00<?, ?B/s]

In [3]:
ds = load_dataset("databricks/databricks-dolly-15k", split="train")

ds = ds.shuffle(seed=42).select(range(2000))

def to_chat(example):
    instr = example["instruction"]
    ctx   = example.get("context") or ""
    resp  = example["response"]
    user  = instr if not ctx else (instr + "\n\n" + ctx)
    messages = [
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user",   "content": user},
        {"role": "assistant","content": resp},
    ]
    example["text"] = tokenizer.apply_chat_template(
        messages, tokenize=False, add_generation_prompt=False
    )
    return example


ds = ds.map(to_chat, remove_columns=ds.column_names)



README.md: 0.00B [00:00, ?B/s]

databricks-dolly-15k.jsonl:   0%|          | 0.00/13.1M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/15011 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

In [4]:
peft_cfg = LoraConfig(
    r=8,
    lora_alpha=16,
    lora_dropout=0.05,
    bias="none",
    target_modules=["q_proj","k_proj","v_proj","o_proj"],
    task_type="CAUSAL_LM",
)

In [7]:
sft_config = TrainingArguments(
    output_dir="qwen25_1p5b_dolly_qlora",
    per_device_train_batch_size=1,
    gradient_accumulation_steps=8,      # effective batch = 8
    num_train_epochs=1,                 # or set max_steps for precise budget
    learning_rate=2e-4,                 # QLoRA-typical LR
    lr_scheduler_type="cosine",
    warmup_ratio=0.03,
    logging_steps=10,
    save_strategy="steps",
    save_steps=100,
    optim="paged_adamw_8bit",           # bitsandbytes optimizer
    bf16=is_bf16,
    fp16=not is_bf16,
    gradient_checkpointing=True,
    max_grad_norm=0.3,
    report_to="none",
)

trainer = SFTTrainer(
    model=model,
    processing_class=tokenizer,
    peft_config=peft_cfg,
    train_dataset=ds,
    args= sft_config,
)

trainer.model.print_trainable_parameters()  # sanity: should be ~0.1–1% of total
trainer.train()

Adding EOS to train dataset:   0%|          | 0/2000 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/2000 [00:00<?, ? examples/s]

Truncating train dataset:   0%|          | 0/2000 [00:00<?, ? examples/s]

The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'bos_token_id': None, 'pad_token_id': 151643}.


trainable params: 2,179,072 || all params: 1,545,893,376 || trainable%: 0.1410


/usr/local/lib/python3.12/dist-packages/torch/_dynamo/eval_frame.py:929: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Step,Training Loss
10,2.449900
20,2.039200
30,1.960700
40,1.882800
50,1.949600
60,1.770600
70,1.704900
80,1.710100
90,1.856600
100,1.886500


/usr/local/lib/python3.12/dist-packages/torch/_dynamo/eval_frame.py:929: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.12/dist-packages/torch/_dynamo/eval_frame.py:929: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


TrainOutput(global_step=250, training_loss=1.8356154327392578, metrics={'train_runtime': 2288.0606, 'train_samples_per_second': 0.874, 'train_steps_per_second': 0.109, 'total_flos': 3016626981488640.0, 'train_loss': 1.8356154327392578, 'epoch': 1.0})

In [19]:
from peft import AutoPeftModelForCausalLM
from transformers import AutoTokenizer, pipeline
import torch, os

# Path to your locally saved adapter folder (note the "./")
adapter_path = "./qwen25_1p5b_dolly_qlora_adapter"

# Load the base model first, then apply the adapter
base_model = "Qwen/Qwen2.5-1.5B-Instruct"

model = AutoPeftModelForCausalLM.from_pretrained(
    adapter_path,
    device_map="auto",
    torch_dtype=torch.float16
)

tokenizer = AutoTokenizer.from_pretrained(base_model, use_fast=True)

pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    torch_dtype=torch.float16,
    device_map="auto"
)

prompt = "Explain the difference between LoRA and QLoRA to a new grad."
messages = [
    {"role": "system", "content": "You are a concise, rigorous mentor."},
    {"role": "user", "content": prompt},
]
text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
out = pipe(text, max_new_tokens=200, do_sample=True, temperature=0.7)
print(out[0]["generated_text"][len(text):])


Device set to use cuda:0


है, मैं आपको कुछ वाक्य पढ़ाने की सलाह देना चाहता हूँ।

आपको एक गुजराती भाषा के बारे में कुछ जानकारी असलिया थी। हालांकि, आपको इस बारे में खुशी हुई है।

मैं यहाँ कुछ उदाहरण शेयर करूंगा:

1. जास्त डॉट: "त्यो धीरे"
2


In [14]:
!ls qwen25_1p5b_dolly_qlora_adapter

ls: cannot access 'qwen25_1p5b_dolly_qlora_adapter': No such file or directory


In [16]:
# Save only the adapters (small!) – quickest for sharing and inference
trainer.model.save_pretrained("qwen25_1p5b_dolly_qlora_adapter")
tokenizer.save_pretrained("qwen25_1p5b_dolly_qlora_adapter")

# Optional: merge LoRA into full fp16 weights (use CPU to avoid OOM)
from peft import PeftModel
base = AutoModelForCausalLM.from_pretrained(MODEL_ID, torch_dtype=torch.float16, device_map=None)
merged = PeftModel.from_pretrained(base, "qwen25_1p5b_dolly_qlora_adapter").merge_and_unload()
merged.save_pretrained("qwen25_1p5b_dolly_qlora_merged", safe_serialization=True)